# User Collaborative Filtering Bagging

In [1]:
import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()

In [2]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [3]:
recommender_list=list()
URM_list=list()
num_recs=20
num_nnz=URM_all.nnz
URM_all_coo=URM_all.tocoo()

for i in range(num_recs):
    
    URM_i=np.zeros(URM_all.shape)
    
    for i in range(num_nnz):
        
        sample_index=np.random.randint(num_nnz)
        user_id = URM_all_coo.row[sample_index]
        item_id = URM_all_coo.col[sample_index]
        rating = URM_all_coo.data[sample_index]
        URM_i[user_id, item_id]=rating
    
    URM_i_csr=sps.csr_matrix(URM_i)
    URM_list.append(URM_i_csr)
    recommender = UserKNNCFRecommender(URM_i_csr)
    recommender.fit(topK=100, shrink=50, similarity='cosine')
    recommender_list.append(recommender)

UserKNNCFRecommender: URM Detected 250 (3.15 %) cold users.
UserKNNCFRecommender: URM Detected 4113 (15.83 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 7947 ( 100 % ), 3946.31 column/sec, elapsed time 0.03 min
UserKNNCFRecommender: URM Detected 243 (3.06 %) cold users.
UserKNNCFRecommender: URM Detected 4110 (15.82 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 7947 ( 100 % ), 4235.76 column/sec, elapsed time 0.03 min
UserKNNCFRecommender: URM Detected 256 (3.22 %) cold users.
UserKNNCFRecommender: URM Detected 4107 (15.81 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 7947 ( 100 % ), 4167.70 column/sec, elapsed time 0.03 min
UserKNNCFRecommender: URM Detected 226 (2.84 %) cold users.
UserKNNCFRecommender: URM Detected 4081 (15.71 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 7947 ( 100 % ), 43

In [4]:
def ensemble_recommend (recommender_list, user, nitems, at):
    """compute recommendation of ensemble recommenders"""
    
    recommendations = np.column_stack((list(range(nitems)),np.zeros(nitems)))
    
    for recommender in recommender_list:

            rec_i=recommender.recommend(user)[:at]

            for rank, item in enumerate(rec_i):
            
                recommendations[item][1]+= at-rank
                
    recommendations=sorted(recommendations, key=lambda item: item[1],reverse=True)
    
    return [int(r[0]) for r in recommendations[:at]]

In [5]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)
for uid in target_user_list:
    recommendation_list=list(map(str,ensemble_recommend(recommender_list,uid,URM_all.shape[1],10)))
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(recommendation_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("CF_bagging_recomm_100_50_cosine_20_learner_nosum.csv")